<center>
<h1>
<h1>APM 53674: ALTeGraD</h1>
<h2>Lab Session 4: Distillation and Retrieval Augmented Generation</h2>
<h4>Lecture: Dr. Guokan Shang<br>
Lab: Yang Zhang and Xiao Fei</h4>
<h5>Tuesday, November 18, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit <a href='https://forms.gle/9dyaes6dimfvyjwq6' target="_blank">here</a> a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>Novemver 23
, 2025 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>

# Install Requirements

In [1]:
# Install required dependencies
!pip -q install torch tqdm jsonlines h5py
!pip -q install --upgrade transformers accelerate vllm
!pip install jedi
# !pip -q install datasets==2.21.0 pandas==2.2.2
# !pip -q install chromadb==0.4.22
# !pip -q install "numpy<2.0" --force-reinstall

# Part 1 - Model Distillation

> In this section, you’ll learn the difference between **white-box** and **black-box** distillation, generate **synthetic data** to train a **student model**, and implement **white-box distillation** to specialize a model for a **RAG on Wikipedia** use case.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:</br>
Explain briefly the difference between black-box and white-box distillation? </br> What are the advantages and inconvenients of each approach?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>
In the black box process, we do not have access to the reasoning of the teacher model but only the answer and our model tries to mimic this answer.
Pros: It is easier to implement.
Cons: Less efficient, does not transfer well on different, more complex problems, recquires more data to train.

In the white box process, we have access to the reasoning of the teacher model and the student model tries to learn this and deduce from this reasoning the answer. \
Pros: It transfers well. It is data efficient, with less data it learns better as more information is transfered by the teacher. \
Cons: Implementation is more complex as you must align the student's internal states or outputs with the teacher's reasoning traces.

<hr style="border:10px solid green"> </hr>
</font></h4>


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 2:</br>
What is the main requirement for a teacher/student pair of models to perform white-box distillation?
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>

As the main objective of a white box distillation is to teach to the student model to learn how, learn the reasoning leading to the answer.

So the main recquirement is to get access to the teacher reasoning.

This requirement rules out using proprietary or closed-source models (like GPT-4 via a standard API) as teachers for white-box distillation. It is only feasible with open-weight models where you have complete access to the internal computations and outputs.

<hr style="border:10px solid green"> </hr>
</font></h4>


## 1.1 - Synthetic Data Generation

We're going to specialize a small 0.5B parameter model to perform RAG by distilling the abilities of a 7B parameter one.

For that we'll be using `Qwen/Qwen2.5-0.5B-Instruct` as student and `Qwen/Qwen2.5-7B-Instruct-AWQ` (quantized version of `Qwen2.5-7B-Instruct`) as teacher.  

In order to perform white-box distillation on generated answers we have two choices.

1. We can perform a forward pass with the teacher, a forward pass with student on the complete sequence, and backprop difference of logprobs using KL Loss.
2. Generation of samples with the teacher, save the logprobs and perform finetuning in a second step.

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 3:</br>
What are the computational advantages of 1. vs 2.?

For 1.:
We get a memory efficient environeent sinc we only need to store the teacher's logprobs in memory during the forward pass, rather than saving them to disk and reloading. This avoids the storage overhead of caching large tensors of logprobs for the entire dataset.

For 2.:
The major computational advantage of Method 2 (generating samples first) is that it is far more cost-effective. The expensive 7B teacher model only performs one forward pass to generate all its reasoning traces and answers, creating a reusable dataset. This means the subsequent, iterative training process only runs the tiny 0.5B student model, completely avoiding the massive computational cost of repeatedly running the large teacher during every training step or epoch. This one-time teacher computation also enables faster experimentation, as different student training strategies can be tested without ever needing to regenerate the teacher's data.
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 3: </b><br>
Your answer here.
<hr style="border:10px solid green"> </hr>
</font></h4>

We're going to generate a bunch of questions related to wikipedia paragraphs.
For that we need to establish a system prompt that will allow for easy extraction.

In [2]:
system_prompt = """
You are a question generator.
The user will provide:

```json
{"title": "the title of an article", "paragraph": "a paragraph from that article"}
```

Your task:

* Generate one clear, self-contained question that can be answered using only the provided paragraph.
* The question must be **specific**, **unambiguous**, and directly tied to the paragraph’s content.
* Return the result with the question as a valid JSON** in the form:

```json
{
  "question": "your question here"
}
```

Example:
User input:

```json
{
"title": "The Moon Landing",
"paragraph": "On July 20, 1969, Neil Armstrong became the first human to set foot on the Moon, followed by Buzz Aldrin."
}
```
Assistant output:

```json
{
  "question": "Who was the first human to set foot on the Moon during the Apollo 11 mission?"
}
```
"""

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 4:</br>
In this system prompt, we don't generate answers, only questions. Explain why it's necessary in the context of white-box distillation.
<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>
In white-box distillation, the teacher model's primary role is to generate the reasoning process and the final answer, not just the question. By only generating the question here, we are creating the input for a subsequent step where the teacher model will be prompted to answer its own generated question while showing its full chain-of-thought reasoning. This separate, dedicated generation of the answer+reasoning is necessary because it allows us to capture the teacher's complete internal logic and token-by-token reasoning traces, which are the essential targets the student model must learn to replicate in white-box distillation.
<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 5:</br>
For synthetic data generation we'll be using vLLM.
vLLM is an optimized llm inference engine that can improve generation speed thanks to hardware specific optimization and computational tricks such as Prefix KV Caching.
(https://docs.vllm.ai/en/latest/features/automatic_prefix_caching.html)</br></br> 1. Explain why prefix caching will be very efficient in our case? </br></br>
2. What sampling `temperature` should we use? Justify.



<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Since the task for processing paragraph in wikipedia are repetitive, automati prefix caching allows our model to store the information of the paragraph and an answer can immediately be generated from this paragraph sikiping the expensive cost of of the context and making our model to generate data as dast as a generation process.


We should choose a low temperature since it  focuses on the highest-probability tokens allowing to get consistent and robust reasoning traces. This would provide a good target for the student. However a slightly higer temperature would allow our model to be more resistent since it would introduce more diversity.

<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Optimized Generation of Synthetic Questions

Complete the below code with the adequate options (prefix caching and `temperature`)

<hr style="border:10px solid blue"> </hr>
</font></h4>


In [3]:
from vllm import LLM
from vllm import SamplingParams
import torch

# Check current GPU memory usage
print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"Allocated GPU memory: {torch.cuda.memory_allocated(0) / 1e9:.1f} GB")
print(f"Cached GPU memory: {torch.cuda.memory_reserved(0) / 1e9:.1f} GB")

# Clear any cached memory
torch.cuda.empty_cache()

print(f"After cleanup - Allocated: {torch.cuda.memory_allocated(0) / 1e9:.1f} GB")
print(f"After cleanup - Cached: {torch.cuda.memory_reserved(0) / 1e9:.1f} GB")

# Try loading with optimized settings
path_teacher = "Qwen/Qwen2.5-7B-Instruct-AWQ"

try:
    llm = LLM(
        model=path_teacher,
        gpu_memory_utilization=0.7,  # Conservative memory usage
        max_model_len=4096,  # Reasonable context length
        trust_remote_code=True,  # Essential for Qwen models
        enforce_eager=True,  # Avoid graph compilation issues
        max_num_batched_tokens=4096,  # Limit batch tokens
        max_num_seqs=4,  # Limit concurrent sequences
        quantization="awq",  # Explicitly specify AWQ
        dtype="half"  # Use half precision
    )
    print("✅ Successfully loaded Qwen2.5-7B-Instruct-AWQ model!")

except Exception as e:
    print(f"❌ AWQ model failed: {e}")

    # Try the non-quantized version with memory optimization
    try:
        path_teacher = "Qwen/Qwen2.5-7B-Instruct"
        llm = LLM(
            model=path_teacher,
            gpu_memory_utilization=0.8,
            max_model_len=2048,  # Reduced context for memory
            trust_remote_code=True,
            enforce_eager=True,
            max_num_seqs=2,  # Very conservative
            dtype="auto"  # Let vLLM choose optimal dtype
        )
        print("✅ Successfully loaded non-quantized Qwen2.5-7B-Instruct model!")

    except Exception as e2:
        print(f"❌ Non-quantized model failed: {e2}")

        # Last resort: Try 1.5B model
        try:
            path_teacher = "Qwen/Qwen2.5-1.5B-Instruct"
            llm = LLM(
                model=path_teacher,
                gpu_memory_utilization=0.8,
                max_model_len=4096,
                trust_remote_code=True
            )
            print("✅ Successfully loaded Qwen2.5-1.5B-Instruct model (smaller but functional)!")

        except Exception as e3:
            print(f"❌ All GPU attempts failed: {e3}")
            raise

# Sampling parameters for question generation
sampling_params = SamplingParams(
    temperature=0.7,
    max_tokens=400,
    stop=None
)

print("🎯 Model and sampling parameters ready!")

Total GPU memory: 15.8 GB
Allocated GPU memory: 0.0 GB
Cached GPU memory: 0.0 GB
After cleanup - Allocated: 0.0 GB
After cleanup - Cached: 0.0 GB
INFO 11-21 15:33:17 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'half', 'max_model_len': 4096, 'gpu_memory_utilization': 0.7, 'max_num_batched_tokens': 4096, 'max_num_seqs': 4, 'disable_log_stats': True, 'quantization': 'awq', 'enforce_eager': True, 'model': 'Qwen/Qwen2.5-7B-Instruct-AWQ'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

INFO 11-21 15:33:49 [model.py:631] Resolved architecture: Qwen2ForCausalLM
INFO 11-21 15:33:49 [model.py:1745] Using max model len 4096
INFO 11-21 15:33:53 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=4096.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

WARNING 11-21 15:33:57 [system_utils.py:103] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 11-21 15:40:13 [llm.py:352] Supported tasks: ['generate']
✅ Successfully loaded Qwen2.5-7B-Instruct-AWQ model!
🎯 Model and sampling parameters ready!


In [4]:
# from vllm import LLM
# from vllm import SamplingParams
# import os
# path_teacher = "Qwen/Qwen2.5-7B-Instruct-AWQ"
# llm = LLM(model=path_teacher, gpu_memory_utilization=0.9, max_model_len=5000, ) # To Complete
# sampling_params = SamplingParams(temperature = 0.7, max_tokens=400) # To Complete


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Question Generation</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

We're going to use the llm.chat vLLM api to generate our samples.
Make the adequate code to generate the data and save it in a questions.jsonl file.

Entries of jsonl file should look like:
```json
{
  "id_doc": <wikipedia_article_id>,
  "id_paragraph": <paragraph_dataset_id>,
  "question": <generated_question>,
  "title": <title_wikipedia_article>,
  "paragraph": <text_of_paragraph>
}
```

You should:

1. Complete `def extract_question(generated_text: str) -> str:` to extract the generated question.
2. Complete `def conversation_generator` to output a generator directly ingestible by `llm.chat` api
3. Complete the dataloader and for loop to generate samples by batches of 4
4. Save the generated questions in a `questions.jsonl` file

In [5]:
import json
import jsonlines
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm.auto import tqdm
from typing import Iterable, Iterator, List, Dict, Any

def extract_question(generated_text: str) -> str:
    """Extract the question from the generated text. If the question is not\
    following the right format return None."""

    try:
        # Clean the generated text - remove any markdown code blocks if present
        cleaned_text = generated_text.strip()
        if "```json" in cleaned_text:
            cleaned_text = cleaned_text.replace("```json", "").replace("```", "").strip()

        # Parse the JSON
        j = json.loads(cleaned_text)
        assert "question" in j
        return j["question"]
    except (json.JSONDecodeError, AssertionError, KeyError):
        return None

def conversation_generator_questions(
    entries: Iterable[dict],
    system_prompt: str
) -> Iterator[dict]:
    """Generate the conversation with the model for question generation."""

    for entry in entries:
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Title: {entry['title']}\nParagraph: {entry['paragraph']}"}
        ]
        yield conversation

# Load the dataset
ds = load_dataset("EvanD/Lab4_wikiparagraphs")

# Define system prompt for question generation
system_prompt_questions = """You are a helpful assistant that generates reading comprehension questions based on Wikipedia articles.
Given a title and paragraph from a Wikipedia article, generate one thoughtful question that can be answered from the paragraph.
Return your response in JSON format with a single key "question" containing the generated question.

Example response:
{"question": "What year was the first computer invented?"}"""

# Prepare the conversations for question generation
conversations_questions = list(conversation_generator_questions(ds['train'], system_prompt_questions))

batch_size = 4
dataloader_questions = DataLoader(conversations_questions, batch_size=batch_size, shuffle=False,
                                num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

# Generate questions
print("Generating questions...")
with jsonlines.open("questions.jsonl", "w") as writer:
    for i, batch in tqdm(enumerate(dataloader_questions), total=len(dataloader_questions)):
        outputs = llm.chat(batch, sampling_params=sampling_params, use_tqdm=False)

        for j, output in enumerate(outputs):
            # Calculate the actual index in the dataset
            idx = i * batch_size + j
            if idx < len(ds['train']):
                original_entry = ds['train'][idx]

                q = extract_question(output.outputs[0].text)

                # Only write if we successfully extracted a question
                if q is not None:
                    entry = {
                        "id_doc": original_entry['id'],  # Using 'id' field from dataset
                        "id_paragraph": original_entry['id'],  # Using same id since no separate paragraph id
                        "question": q,
                        "title": original_entry['title'],
                        "paragraph": original_entry['paragraph']
                    }
                    writer.write(entry)

print("Question generation completed!")

Generating questions...


  0%|          | 0/50 [00:00<?, ?it/s]

INFO 11-21 15:40:21 [chat_utils.py:557] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
Question generation completed!


## 1.2 - Logprobs Generation

We get to the second part of this distillation where we are interested in distilling the answers logprobs of our teacher model to specialize our 0.5B model to perform Retrieval Augmented Generation (RAG).

First we're going to generate the logprobs with our 7B parameter model.

We'll use the following system prompt:

In [6]:
system_prompt = (
    "You are an assistant for a Retrieval-Augmented Generation (RAG) system.\n"
    "Answer the question using only the provided documents. "
    "If the answer cannot be found in the provided documents, respond that the answer is not available in the provided document database. "
    "Documents:\n{context_block}\n\n"
)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: Complete Conversation Generator</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>

Based on the previous code you made to generate questions, make a second one to generate answers and saving logprobs. We updated the sampling parameters of vLLM to return the logprobs and the token ids of the 20 most probable tokens.

As this can result in high quantity of data in practice we're going to store generated conversations in a .jsonl file and generated logprobs and token ids to a hdf5 file (see https://docs.h5py.org/en/stable/ for more information on hdf5).

`save_logprobs_hdf5()` is already implemented for you, and allows to save the logprobs to a .h5 hdf5 file, and increments sequences automatically

The structure of the conversation generator `conversation_generator()` function is implemented, you need to complete it.



In [7]:
import h5py, os
import numpy as np

def save_logprobs_hdf5(path, sequences, start_idx=None):
    """
    sequences: list of sequences
       each sequence = list of steps
          each step = {token_id: Logprob(logprob=..., ...), ...}
    """
    mode = "a" if os.path.exists(path) else "w"
    with h5py.File(path, mode) as f:
        # choose index to start writing
        if start_idx is None:
            # auto-continue numbering if file already has data
            existing = [int(k.split("_")[1]) for k in f.keys() if k.startswith("seq_")]
            start_idx = max(existing)+1 if existing else 0

        for s_i, seq in enumerate(sequences, start=start_idx):
            g = f.create_group(f"seq_{s_i}")
            for t_i, step in enumerate(seq):
                token_ids = np.fromiter(step.keys(), dtype=np.int32)
                logprobs  = np.array([lp.logprob for lp in step.values()], dtype=np.float32)
                g.create_dataset(f"step_{t_i}/token_ids", data=token_ids, compression="gzip")
                g.create_dataset(f"step_{t_i}/logprobs",  data=logprobs,  compression="gzip")





In [8]:
import h5py
import numpy as np
import os

def conversation_generator_answers(
    entries: Iterable[dict],
    system_prompt: str
) -> Iterator[dict]:
    """Generate the conversation with the model for answer generation."""

    for entry in entries:
        # Format the context block with the paragraph
        context_block = entry['paragraph']

        # Format the system prompt with the context
        formatted_system_prompt = system_prompt.format(context_block=context_block)

        conversation = [
            {"role": "system", "content": formatted_system_prompt},
            {"role": "user", "content": entry['question']}
        ]
        yield conversation

def save_logprobs_hdf5(path, sequences, start_idx=None):
    """
    sequences: list of sequences
       each sequence = list of steps
          each step = {token_id: Logprob(logprob=..., ...), ...}
    """
    mode = "a" if os.path.exists(path) else "w"
    with h5py.File(path, mode) as f:
        # choose index to start writing
        if start_idx is None:
            # auto-continue numbering if file already has data
            existing = [int(k.split("_")[1]) for k in f.keys() if k.startswith("seq_")]
            start_idx = max(existing)+1 if existing else 0

        for s_i, seq in enumerate(sequences, start=start_idx):
            g = f.create_group(f"seq_{s_i}")
            for t_i, step in enumerate(seq):
                token_ids = np.fromiter(step.keys(), dtype=np.int32)
                logprobs = np.array([lp.logprob for lp in step.values()], dtype=np.float32)
                g.create_dataset(f"step_{t_i}/token_ids", data=token_ids, compression="gzip")
                g.create_dataset(f"step_{t_i}/logprobs", data=logprobs, compression="gzip")

# System prompt for RAG answers
system_prompt_answers = (
    "You are an assistant for a Retrieval-Augmented Generation (RAG) system.\n"
    "Answer the question using only the provided documents. "
    "If the answer cannot be found in the provided documents, respond that the answer is not available in the provided document database. "
    "Documents:\n{context_block}\n\n"
)

# Load the questions we generated earlier
try:
    with jsonlines.open("questions.jsonl", "r") as reader:
        questions_data = list(reader)
    print(f"Loaded {len(questions_data)} questions for answer generation")
except FileNotFoundError:
    print("Questions file not found. Please run the question generation first.")
    questions_data = []

if questions_data:
    # Prepare conversations for answer generation
    conversations_answers = list(conversation_generator_answers(questions_data, system_prompt_answers))

    # Update sampling parameters to include logprobs
    sampling_params_logprobs = SamplingParams(
        temperature=0.7,
        max_tokens=400,
        logprobs=20,  # Get top 20 tokens with logprobs
        stop=None
    )

    batch_size = 4
    dataloader_answers = DataLoader(conversations_answers, batch_size=batch_size, shuffle=False,
                                   num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

    # Generate answers with logprobs
    print("Generating answers with logprobs...")
    with jsonlines.open("rag_conversations.jsonl", "w") as writer:
        all_logprob_sequences = []

        for i, batch in tqdm(enumerate(dataloader_answers), total=len(dataloader_answers)):
            outputs = llm.chat(batch, sampling_params=sampling_params_logprobs, use_tqdm=False)

            for j, output in enumerate(outputs):
                idx = i * batch_size + j
                if idx < len(questions_data):
                    original_entry = questions_data[idx]

                    # Extract the generated answer
                    answer = output.outputs[0].text

                    # Extract logprobs for each generation step
                    logprob_sequence = []
                    if hasattr(output.outputs[0], 'logprobs') and output.outputs[0].logprobs:
                        for step_logprobs in output.outputs[0].logprobs:
                            if step_logprobs is not None:
                                # Convert Logprob objects to dictionary format
                                step_dict = {}
                                for token_id, logprob_obj in step_logprobs.items():
                                    step_dict[token_id] = logprob_obj
                                logprob_sequence.append(step_dict)

                    all_logprob_sequences.append(logprob_sequence)

                    # Save conversation data
                    conversation_entry = {
                        "id_doc": original_entry['id_doc'],
                        "id_paragraph": original_entry['id_paragraph'],
                        "question": original_entry['question'],
                        "title": original_entry['title'],
                        "paragraph": original_entry['paragraph'],
                        "answer": answer,
                        "logprobs_sequence_index": len(all_logprob_sequences) - 1  # Reference to logprobs in HDF5
                    }
                    writer.write(conversation_entry)

        # Save all logprobs to HDF5 file
        if all_logprob_sequences:
            save_logprobs_hdf5("rag_logprobs.h5", all_logprob_sequences)
            print(f"Saved {len(all_logprob_sequences)} logprob sequences to rag_logprobs.h5")

    print("Answer generation with logprobs completed!")

Loaded 198 questions for answer generation
Generating answers with logprobs...


  0%|          | 0/50 [00:00<?, ?it/s]

Saved 198 logprob sequences to rag_logprobs.h5
Answer generation with logprobs completed!


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 6:</br>
When completing conversation_generator we have several ways of sampling.
What are good paragraph sampling strategies we could use to ensure good performance of downstream model?

<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>


According to the course a robust paragraph sampling strategy has to avoid model collapse and performance degeneration must explicitly prioritize diversity and the preservation of "tail" data.
So we have to get a broad coverage of our data and take data from different clusters.
This is why difficulty-based sampling seems to be the best choice here as it seeks for complex sentences,  syntax and vocabulary diversity.
With this paragraph sampling our model should get good performance of downstream model.


<hr style="border:10px solid green"> </hr>
</font></h4>

In [9]:
import random

doc_id_to_paragraphs = {}

for line in ds["train"]:
    doc_id = line["id"]
    paragraph = line["paragraph"]
    title = line["title"]
    if doc_id not in doc_id_to_paragraphs:
        doc_id_to_paragraphs[doc_id] = []
    doc_id_to_paragraphs[doc_id].append(title + " -- " + paragraph)  # We add the title to contextualize the paragraph

def conversation_generator(
    path_jsonl: str,
    system_prompt: str,
    top_k: int = 3
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    with jsonlines.open(path_jsonl, "r") as f:
      for q_p in f:
        number_of_paragraphs_in_context = random.sample(range(1, top_k + 1), 1)[0]
        paragraphs = [q_p["title"] + " -- " + q_p["paragraph"]] # We add the title to contextualize the paragraph
        while len(paragraphs) < number_of_paragraphs_in_context:
          # With 50% probability, add a paragraph from the same article
          if random.random() < 0.5 and q_p["id_doc"] in doc_id_to_paragraphs:
            # Add another paragraph from the same document
            same_doc_paragraphs = doc_id_to_paragraphs[q_p["id_doc"]]
            if len(same_doc_paragraphs) > 1:
                available_paragraphs = [p for p in same_doc_paragraphs if p not in paragraphs]
                if available_paragraphs:
                    paragraphs.append(random.choice(available_paragraphs))
          else:
            # Add a paragraph from a different article
            other_doc_ids = [doc_id for doc_id in doc_id_to_paragraphs.keys() if doc_id != q_p["id_doc"]]
            if other_doc_ids:
                random_doc_id = random.choice(other_doc_ids)
                random_paragraph = random.choice(doc_id_to_paragraphs[random_doc_id])
                paragraphs.append(random_paragraph)

        random.shuffle(paragraphs)
        system_prompt_formatted = system_prompt.format(
            context_block="\n\n".join(f"[Document {i+1}]: {doc}" for i, doc in enumerate(paragraphs))
        )
        conversation = [
            {"role": "system", "content": system_prompt_formatted},
            {"role": "user", "content": q["question"]}
        ]
        yield conversation



In [10]:
import random

doc_id_to_paragraphs = {}

for line in ds["train"]:
    doc_id = line["id"]
    paragraph = line["paragraph"]
    title = line["title"]
    if doc_id not in doc_id_to_paragraphs:
        doc_id_to_paragraphs[doc_id] = []
    doc_id_to_paragraphs[doc_id].append(title + " -- " + paragraph)  # We add the title to contextualize the paragraph

def conversation_generator(
    path_jsonl: str,
    system_prompt: str,
    top_k: int = 3
    ) -> Iterator[dict]:

    """Generate the conversation with the model."""

    with jsonlines.open(path_jsonl, "r") as f:
      for q_p in f:
        number_of_paragraphs_in_context = random.sample(range(1, top_k + 1), 1)[0]
        paragraphs = [q_p["title"] + " -- " + q_p["paragraph"]] # We add the title to contextualize the paragraph

        while len(paragraphs) < number_of_paragraphs_in_context:
          # With 50% probability, add a paragraph from the same article
          if random.random() < 0.5 and q_p["id_doc"] in doc_id_to_paragraphs:
            # Add another paragraph from the same document
            same_doc_paragraphs = doc_id_to_paragraphs[q_p["id_doc"]]
            if len(same_doc_paragraphs) > 1:
                available_paragraphs = [p for p in same_doc_paragraphs if p not in paragraphs]
                if available_paragraphs:
                    paragraphs.append(random.choice(available_paragraphs))
          else:
            # Add a paragraph from a different article
            other_doc_ids = [doc_id for doc_id in doc_id_to_paragraphs.keys() if doc_id != q_p["id_doc"]]
            if other_doc_ids:
                random_doc_id = random.choice(other_doc_ids)
                random_paragraph = random.choice(doc_id_to_paragraphs[random_doc_id])
                paragraphs.append(random_paragraph)

        random.shuffle(paragraphs)
        system_prompt_formatted = system_prompt.format(
            context_block="\n\n".join(f"[Document {i+1}]: {doc}" for i, doc in enumerate(paragraphs))
        )
        conversation = [
            {"role": "system", "content": system_prompt_formatted},
            {"role": "user", "content": q_p["question"]}  # Fixed: changed 'q' to 'q_p'
        ]
        yield conversation

from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from typing import Iterable

# Define the system prompt
system_prompt = (
    "You are an assistant for a Retrieval-Augmented Generation (RAG) system.\n"
    "Answer the question using only the provided documents. "
    "If the answer cannot be found in the provided documents, respond that the answer is not available in the provided document database. "
    "Documents:\n{context_block}\n\n"
)

batch_size = 4

conversations = list(conversation_generator(path_jsonl="questions.jsonl", system_prompt=system_prompt))

dataloader = DataLoader(conversations, batch_size=batch_size, shuffle=False, num_workers=2, prefetch_factor=2, collate_fn=lambda x: x)

sampling_params = SamplingParams(temperature=0.2, max_tokens=400, logprobs=20)

with jsonlines.open("conversations_rag.jsonl", "w") as writer:
    for batch in tqdm(dataloader):
        out = llm.chat(batch,
                        sampling_params=sampling_params,
                        use_tqdm=False)
        for i, b in enumerate(batch):
            text = out[i].outputs[0].text
            b.append({"role": "assistant", "content": text})
            writer.write(b)
        save_logprobs_hdf5("logprobs.h5", [out[i].outputs[0].logprobs for i in range(len(out))])

  0%|          | 0/50 [00:00<?, ?it/s]

## 1.3 - KL-Divergence and Distillation

**You should restart the notebook kernel to free the gpu memory from the 7B model that is no longer needed**

Now that we have the generated conversations and their logprobs we can train our 0.5B model to output the same distribution.

The attentive student would have noticed that we have an incomplete representation of the probability distribution over tokens due to only keeping the top-20 logprobs.


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 7:</br>
What are the two solutions you can see to approximate full distillation despite only having the top-20 logprobs?

<hr style="border:10px solid red"> </hr>  
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>
The first solution would be:
Hierarchical distillation: teacher's top-20 distribution is used directly for the high-probability tokens and we employ a temperature-scaled uniform prior for the remaining vocabulary.
Doing this we ensure the student learns both the precise high-likelihood targets and a smoothed representation of lower-probability options.

The second solution would be:
Using contrive learning with negative sampling
This approach allows us to treat the top-20 tokens as positive examples and randomly sampling from the remaining vocabulary as negatives, training the student model to distinguish between high-probability teacher outputs and impossible alternatives.
Hence it approximates well the full distribution through contrastive discrimination rather than exact probability matching.

<hr style="border:10px solid green"> </hr>
</font></h4>

We supply two functions to help in this implementation:

- `find_subsequence()`that allows to find all the occurences of a token subsequence in a 1-D tensor
- `get_labels()` that allows to expand the top-20 logprobs to the whole vocabulary, implicitly setting prob to zero for other tokens
- `QwenKLDataset`that loads samples from .h5 and .jsonl files, remove problematic inconsistent tokenized examples and outputs samples tokenized for training.

To simplify we will train with a batch size of 1, you can implement gradient accumulation if you wish.

The `PREFIX_ASSISTANT`variable contains the tokens that encode for

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [12]:
import torch

def find_subsequence(input_ids: torch.Tensor, subseq: torch.Tensor):
    """Find the first index of each occurence of a subsequence in the input_ids."""

    subseq_len = len(subseq)
    matches = []
    for idx in range(input_ids.size(0) - subseq_len + 1):
        if torch.equal(input_ids[idx:idx + subseq_len], subseq):
            return [idx]
    return []

def get_labels(logprobs: torch.Tensor, tokens, size_vocab: int = 151936, offset: int = 0):
    """Get the greedy max probability tokenized sequence from the logprobs."""
    labels = torch.full((len(logprobs), size_vocab), torch.finfo(torch.float16).min, dtype=torch.float16)
    for idx, logprobs_token in enumerate(logprobs):
        for token_id, logprob in zip(tokens[idx], logprobs_token):
            labels[idx][token_id] = logprob
    return labels

In [13]:
from torch.utils.data import Dataset
import jsonlines
import h5py


PREFIX_ASSISTANT = [198, 151644, 77091, 198]

class QwenKLDataset(Dataset):
    """Dataset for finetuning Qwen model with KL divergence loss."""
    def __init__(
            self,
            path_h5,
            path_jsonl
        ):
        self.entries = []
        with jsonlines.open(path_jsonl, "r") as reader:
            with h5py.File(path_h5, "r") as f:
                for j, line in tqdm(enumerate(reader)):
                    inputs = tokenizer.apply_chat_template(line, add_generation_prompt=False, tokenize=True, return_dict=True, return_tensors="pt")
                    idx_subseq = find_subsequence(inputs["input_ids"][0], subseq = torch.tensor(PREFIX_ASSISTANT))[0]
                    seq_f = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][0] for i in range(len(f[f"seq_{j}"]))]
                    try:
                        assert len(inputs["input_ids"][0][idx_subseq+3:-2]) == len(seq_f)
                    except AssertionError:
                        print(f"AssertionError {j}, skipping inconsistent detokenization/tokenization")
                    tokens = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][:] for i in range(len(f[f"seq_{j}"]))]
                    logprobs = [f[f"seq_{j}"][f"step_{i}"]["logprobs"][:] for i in range(len(f[f"seq_{j}"]))]
                    inputs["input_ids"] = inputs["input_ids"].cuda()
                    self.entries.append(
                        {
                            "inputs": inputs,
                            "idx_subseq": idx_subseq + 3,
                            "seq_len": len(seq_f),
                            "logprobs": torch.tensor(np.array(logprobs)),
                            "tokens": torch.tensor(np.array(tokens))
                        }
                    )

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        """
        Return for a given entry:

        new_tokens: torch.Tensor, the tokenized conversation with the logprobs of the assistant answer inserted.
        idx_match: int, the index at which the assistant answer starts in the new_tokens.
        len_logprob_sequence: int, the tokenized length of the assistant answer.
        labels: torch.Tensor, the logprobs of the assistant answer for each token.
        """
        entry = self.entries[idx]
        entry["labels"] = get_labels(entry["logprobs"], entry["tokens"], offset=0, size_vocab=151936).cuda()

        return entry

KL-Divergence is a metric often used to quantify the difference of probability mass between two distributions.
It's not mathematically defined as a distance because of its asymetric nature:

$$D_{KL}(P \parallel Q) = \sum_{x} P(x) \log \frac{P(x)}{Q(x)}$$

In knowledge distillation, we often minimize the **Kullback–Leibler divergence** between the teacher’s output distribution $P_T$ and the student’s output distribution $P_S$.

$$
D_{KL}(P_T \parallel P_S) = \sum_x P_T(x) \log \frac{P_T(x)}{P_S(x)}
$$
$$
D_{KL}(P_S \parallel P_T) = \sum_x P_S(x) \log \frac{P_S(x)}{P_T(x)}
$$


<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 8:</br>
What qualitative difference would it make if we minimize $D_{KL}(P_T \parallel P_S)$ instead of $D_{KL}(P_S \parallel P_T)$? How would it affect the convergence of the student distribution?

<hr style="border:10px solid red"> </hr>  
</font></h4>



<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 8: </b><br>

Minimizing $D_{KL}(P_T \parallel P_S)$ (forward KL) increases the weigh, importance given to the regions where the teacher has a non zero probability which ensures that our model will capture all the teachers mode but may include very low probability regions.

In contrast, minimizing $D_{KL}(P_S \parallel P_T)$ (reverse KL) the student focus on the teacher's highest probability modes, ignoring sometimes low probaility regions of the teacher's modes allowing a faster convergence.
However, the student might sometimes just giving the most probable response instead of having the full diversity of the teacher model.

If we search for a fast model we would do reverse KL but if we search for an accurate model we would do forward KL.
<hr style="border:10px solid green"> </hr>
</font></h4>

**You need to ensure you restarted the kernel and have sufficiently free memory (no 7B model running)**

check with `! nvidia-smi`

In [14]:
! nvidia-smi

Fri Nov 21 15:49:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             29W /   70W |   11326MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----




This code is very basic, and allows you to test a training over the small number of samples you've generated. Due to limitations of Google Colab we cannot go much further, but this first part should have given you the basics on how to perform white-box distillation.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: Complete The Training Code</br>

<hr style="border:10px solid blue"> </hr>
</font></h4>


- Loss function
- gradient accumulation handling



In [15]:
!pip install jsonlines

In [2]:
# Install requirements (run this first)
!pip -q install torch tqdm jsonlines h5py
!pip -q install --upgrade transformers accelerate
!pip install jedi

In [3]:
from torch.utils.data import Dataset
import jsonlines
import h5py
from transformers import AutoTokenizer
from tqdm.auto import tqdm
import torch
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

PREFIX_ASSISTANT = [198, 151644, 77091, 198]

def find_subsequence(input_ids: torch.Tensor, subseq: torch.Tensor):
    """Find the first index of each occurence of a subsequence in the input_ids."""
    subseq_len = len(subseq)
    matches = []
    for idx in range(input_ids.size(0) - subseq_len + 1):
        if torch.equal(input_ids[idx:idx + subseq_len], subseq):
            return [idx]
    return []

def get_labels(logprobs: torch.Tensor, tokens, size_vocab: int = 151936, offset: int = 0):
    """Get the greedy max probability tokenized sequence from the logprobs."""
    labels = torch.full((len(logprobs), size_vocab), torch.finfo(torch.float16).min, dtype=torch.float16)
    for idx, logprobs_token in enumerate(logprobs):
        for token_id, logprob in zip(tokens[idx], logprobs_token):
            labels[idx][token_id] = logprob
    return labels

class QwenKLDataset(Dataset):
    """Dataset for finetuning Qwen model with KL divergence loss."""
    def __init__(
            self,
            path_h5,
            path_jsonl
        ):
        self.entries = []
        with jsonlines.open(path_jsonl, "r") as reader:
            with h5py.File(path_h5, "r") as f:
                for j, line in tqdm(enumerate(reader)):
                    inputs = tokenizer.apply_chat_template(line, add_generation_prompt=False, tokenize=True, return_dict=True, return_tensors="pt")
                    idx_subseq = find_subsequence(inputs["input_ids"][0], subseq = torch.tensor(PREFIX_ASSISTANT))[0]
                    seq_f = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][0] for i in range(len(f[f"seq_{j}"]))]
                    try:
                        assert len(inputs["input_ids"][0][idx_subseq+3:-2]) == len(seq_f)
                    except AssertionError:
                        print(f"AssertionError {j}, skipping inconsistent detokenization/tokenization")
                        continue
                    tokens = [f[f"seq_{j}"][f"step_{i}"]["token_ids"][:] for i in range(len(f[f"seq_{j}"]))]
                    logprobs = [f[f"seq_{j}"][f"step_{i}"]["logprobs"][:] for i in range(len(f[f"seq_{j}"]))]
                    inputs["input_ids"] = inputs["input_ids"].cuda()
                    self.entries.append(
                        {
                            "inputs": inputs,
                            "idx_subseq": idx_subseq + 3,
                            "seq_len": len(seq_f),
                            "logprobs": torch.tensor(np.array(logprobs)),
                            "tokens": torch.tensor(np.array(tokens))
                        }
                    )

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        """
        Return for a given entry:
        new_tokens: torch.Tensor, the tokenized conversation with the logprobs of the assistant answer inserted.
        idx_match: int, the index at which the assistant answer starts in the new_tokens.
        len_logprob_sequence: int, the tokenized length of the assistant answer.
        labels: torch.Tensor, the logprobs of the assistant answer for each token.
        """
        entry = self.entries[idx]
        entry["labels"] = get_labels(entry["logprobs"], entry["tokens"], offset=0, size_vocab=151936).cuda()
        return entry

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np

# Complete the model and dataset initialization
student_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", torch_dtype=torch.float16).cuda()
dataset = QwenKLDataset("logprobs.h5", "conversations_rag.jsonl")

num_epochs = 2
grad_accum_steps = 8
optimizer = torch.optim.AdamW(student_model.parameters(), lr=5e-5)
loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=lambda x: x[0])
num_training_steps = num_epochs * len(loader) // grad_accum_steps
scheduler = get_linear_schedule_with_warmup(optimizer, 0, num_training_steps)

# KL Divergence loss function
loss_fn = torch.nn.KLDivLoss(reduction='batchmean')

student_model.train()
total_loss = 0
accumulation_steps = 0

for epoch in range(num_epochs):
    for i, batch in enumerate(loader):
        outputs = student_model(batch["inputs"]["input_ids"])

        # Get the logits for the assistant response part only
        start_idx = batch["idx_subseq"]
        end_idx = start_idx + batch["seq_len"]
        student_logits = outputs.logits[0, start_idx:end_idx, :]

        # Convert teacher logprobs to probabilities
        teacher_probs = torch.softmax(batch["labels"], dim=-1)

        # Convert student logits to log probabilities
        student_log_probs = torch.log_softmax(student_logits, dim=-1)

        # Compute KL divergence loss
        loss = loss_fn(student_log_probs, teacher_probs)
        loss = loss / grad_accum_steps
        loss.backward()

        total_loss += loss.item() * grad_accum_steps
        accumulation_steps += 1

        if (i + 1) % grad_accum_steps == 0 or (i + 1) == len(loader):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            batch_loss = total_loss / accumulation_steps
            print(f"Epoch {epoch+1}, step {i+1}/{len(loader)}: loss = {batch_loss:.4f}")
            total_loss = 0
            accumulation_steps = 0

student_model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")

0it [00:00, ?it/s]

Epoch 1, step 8/198: loss = 0.2656
Epoch 1, step 16/198: loss = nan
Epoch 1, step 24/198: loss = nan
Epoch 1, step 32/198: loss = nan
Epoch 1, step 40/198: loss = nan
Epoch 1, step 48/198: loss = nan
Epoch 1, step 56/198: loss = nan
Epoch 1, step 64/198: loss = nan
Epoch 1, step 72/198: loss = nan
Epoch 1, step 80/198: loss = nan
Epoch 1, step 88/198: loss = nan
Epoch 1, step 96/198: loss = nan
Epoch 1, step 104/198: loss = nan
Epoch 1, step 112/198: loss = nan
Epoch 1, step 120/198: loss = nan
Epoch 1, step 128/198: loss = nan
Epoch 1, step 136/198: loss = nan
Epoch 1, step 144/198: loss = nan
Epoch 1, step 152/198: loss = nan
Epoch 1, step 160/198: loss = nan
Epoch 1, step 168/198: loss = nan
Epoch 1, step 176/198: loss = nan
Epoch 1, step 184/198: loss = nan
Epoch 1, step 192/198: loss = nan
Epoch 1, step 198/198: loss = nan
Epoch 2, step 8/198: loss = nan
Epoch 2, step 16/198: loss = nan
Epoch 2, step 24/198: loss = nan
Epoch 2, step 32/198: loss = nan
Epoch 2, step 40/198: loss = 

('finetuned_model/tokenizer_config.json',
 'finetuned_model/special_tokens_map.json',
 'finetuned_model/chat_template.jinja',
 'finetuned_model/vocab.json',
 'finetuned_model/merges.txt',
 'finetuned_model/added_tokens.json',
 'finetuned_model/tokenizer.json')

# Part 2 - Retrieval Augmented Generation (RAG)

In this section, we will discuss the concept of **Retrieval-Augmented Generation (RAG)** — a framework that combines **information retrieval** and **language generation**. RAG enables language models to access **external knowledge sources** at inference time, reducing hallucinations and improving factual accuracy.

We will explore how to:
- Build and index a **Vector database** from a corpus (here: Wikipedia sample).
- Retrieve the most relevant documents given a query using **embedding-based similarity**.
- Integrate retrieval results into the **generation pipeline** to produce context-aware answers.


In [1]:
# Run this in the FIRST cell after reset
!pip install --quiet "numpy<2.0" pandas==2.2.2
!pip install --quiet transformers==4.36.2 datasets==2.21.0 torch==2.9.0
!pip install --quiet chromadb==0.4.22

# Now import
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import chromadb

print("All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.11.2 requires tokenizers>=0.21.1, but you have tokenizers 0.15.2 which is incompatible.
vllm 0.11.2 requires transformers<5,>=4.56.0, but you have transformers 4.36.2 which is incompatible.
xgrammar 0.1.25 requires transformers>=4.38.0, but you have transformers 4.36.2 which is incompatible.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

All packages installed successfully!


In [3]:
import os
import json
import random
import torch
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModel

In [4]:
# --- Configuration ---
SEED = 42
NUM_ROWS = 1000
DATA_PATH = "wikipedia_20231101_en_1000.csv"

if os.path.exists(DATA_PATH):
    print(f"✅ Found existing dataset at {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
else:
    print("⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...")
    random.seed(SEED)

    # Load the Wikipedia dataset
    stream_ds = load_dataset(
        "wikimedia/wikipedia",
        "20231101.en",
        split="train",
        streaming=True
    )

    buffered_stream = stream_ds.shuffle(seed=SEED, buffer_size=200_000)

    sampled = []
    for ex in buffered_stream:
        try:
            if int(ex["id"]) % 2 == 0:
                sampled.append(ex)
            if len(sampled) >= NUM_ROWS:
                break
        except:
            continue

    # Create DataFrame
    df = pd.DataFrame(sampled)[["id", "url", "title", "text"]]
    df.to_csv(DATA_PATH, index=False)
    print(f"💾 Dataset saved to {DATA_PATH}")


⏳ Generating new dataset from Wikimedia (English, 2023-11-01)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

💾 Dataset saved to wikipedia_20231101_en_1000.csv


In [5]:
#Display Basic Info ---
print("Sampled shape:", df.shape)
print("Columns:", df.columns.tolist())
display(df.head())

Sampled shape: (1000, 4)
Columns: ['id', 'url', 'title', 'text']


,id,url,title,text
0,64741026,https://en.wikipedia.org/wiki/Indrani%20Perera,Indrani Perera,Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: bor...
1,66847582,https://en.wikipedia.org/wiki/August%20Laur,August Laur,August Laur (9 October 1886 Vana-Põltsamaa Par...
2,66467526,https://en.wikipedia.org/wiki/Daniele%20Solcia,Daniele Solcia,Daniele Solcia (born 7 March 2001) is an Itali...
3,65913988,https://en.wikipedia.org/wiki/Eric%20Takabatake,Eric Takabatake,Eric Takabatake (born 9 January 1991) is a Bra...
4,64723650,https://en.wikipedia.org/wiki/Nafissath%20Radji,Nafissath Radji,Nafissath Radji (born 2 August 2002 in Porto-N...


In [6]:
# Visualize Random Wikipedia Articles ---

NUM_EXAMPLES = 3  # number of random samples to show
samples = df.sample(NUM_EXAMPLES, random_state=random.randint(0, 10000))

for _, row in samples.iterrows():
    display(HTML(f"""
    <hr style="border:2px solid #ccc">
    <h3><b>Title:</b> {row['title']}</h3>
    <p><b>URL:</b> <a href="{row['url']}" target="_blank">{row['url']}</a></p>
    <p style="text-align: justify;"><b>Text:</b><br>{row['text']}</p>
    """))


### **Document Chunking**

The first step in building a RAG pipeline is **chunking**, where large documents are divided into smaller, semantically coherent pieces.  
Chunking allows the retriever to work on manageable text segments instead of entire documents, improving retrieval precision and reducing computational load.  


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: Naive Fixed-Length Chunking  
Split each document into overlapping fixed-length chunks to prepare text for retrieval.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [7]:

def text_splitting(text, chunk_length=300, chunk_overlap=100):
    """
    Splits text into fixed-length chunks with overlap.
    """
    out = []
    stride = chunk_length - chunk_overlap ## FILL THE GAP: define the stride as the effective step between chunks
    for i in range(0, len(text), stride):
        chunk = text[i:i+chunk_length] ## FILL THE GAP: extract a substring of size 'chunk_length' starting at 'i'
        out.append(chunk)
    return out

# Apply to all documents
df["naive_chunks"] = df["text"].apply(lambda t: text_splitting(t, chunk_length=300, chunk_overlap=100))

num_chunks = df["naive_chunks"].apply(len)
print(f"Average number of chunks per document: {num_chunks.mean():.2f}")
print(f"Total number of chunks: {num_chunks.sum()}")

example_idx = 0
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]["title"])
print("Original length:", len(df.iloc[example_idx]["text"]))
print("Number of chunks:", len(df.iloc[example_idx]["naive_chunks"]))
print("\nFirst 2 chunks:\n")
for i, c in enumerate(df.iloc[example_idx]["naive_chunks"][:2]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")

Average number of chunks per document: 11.50
Total number of chunks: 11498

--- Example document ---
Title: Indrani Perera
Original length: 3012
Number of chunks: 16

First 2 chunks:

Chunk 1:
Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: born 15 February), is a Sri Lankan singer and playback singer. Indrani along with Clarence Wijewardena and Annesley Malewana are known as "The Original Sinhala Pop Trio".

Early life 
She was born on 15 February in Borella, and is the second of three girls 
--------------------------------------------------------------------------------
Chunk 2:
la Pop Trio".

Early life 
She was born on 15 February in Borella, and is the second of three girls in the family. Her father, Abeypala Perera was a Buddhist and mother, Muriel Perera was a Christian. She has one elder sister, Mallika and one younger sister, Iranganie. Indrani studied at  Presbyteri
--------------------------------------------------------------------------------


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: Paragraph-Aware Chunking  
Implement a smarter chunking strategy by using the ('.') as a boundary to split text into sentences or short paragraphs, then regroup them until reaching the desired chunk length.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [8]:
def text_splitting_paragraph(text, chunk_length=300):
    """
    Splits text by sentences/paragraphs (using '.' as boundary)
    and groups them until reaching the desired chunk length.
    """
    out = []
    paragraph_list = text.split('. ')  # FILLED: split text using '. ' as separator
    current_text = ""
    length = 0
    for par in paragraph_list:
        if length != 0 and length + len(par) < chunk_length:
            current_text += '. ' + par  # FILLED: extend chunk with next segment
            length += len(par) + 2  # FILLED: increment length (including '. ')
        else:
            if len(current_text) != 0:
                out.append(current_text)  # FILLED: store completed chunk
            current_text = par  # FILLED: start new chunk with current paragraph
            length = len(par)  # FILLED: reset length counter
    if len(current_text) > 0:
        out.append(current_text)  # FILLED: add last remaining chunk
    return out


# Apply to all documents
df["paragraph_chunks"] = df["text"].apply(lambda t: text_splitting_paragraph(t, chunk_length=300))

# Compute stats
num_chunks_par = df["paragraph_chunks"].apply(len)
print(f"Average number of paragraph-based chunks per document: {num_chunks_par.mean():.2f}")
print(f"Total number of paragraph-based chunks: {num_chunks_par.sum()}")

# Example comparison
example_idx = 432 #Check out other examples
print("\n--- Example document ---")
print("Title:", df.iloc[example_idx]['title'])
print("Original length:", len(df.iloc[example_idx]['text']))
print(f"Character based chunks: {len(df.iloc[example_idx]['naive_chunks'])}")
print(f"Paragraph based chunks: {len(df.iloc[example_idx]['paragraph_chunks'])}")

print("\nParagraph chunk preview:\n")
for i, c in enumerate(df.iloc[example_idx]['paragraph_chunks'][:6]):
    print(f"Chunk {i+1}:\n{c[:400]}\n{'-'*80}")

Average number of paragraph-based chunks per document: 7.55
Total number of paragraph-based chunks: 7551

--- Example document ---
Title: John Ballantine (banker)
Original length: 12204
Character based chunks: 62
Paragraph based chunks: 32

Paragraph chunk preview:

Chunk 1:
John Ballantine (1743–1812), was a Scottish merchant and banker and one of the greatest friends, admirers and closest confidants of Robert Burns
--------------------------------------------------------------------------------
Chunk 2:
 Significantly Ballantine gave the poet advice on the selection of poems for his First Kilmarnock Edition as well as being asked for his opinion on the bard's poems.

Life and character
John was born in Ayr to William Ballantine, a baillie in Ayr and his mother was Elizabeth Bowman
--------------------------------------------------------------------------------
Chunk 3:
He was a merchant and a Banker and in 1787 he became the Provost of Ayr, during which time he helped establish Ayr A

<b><h4><font color='red'>

<hr style="border:10px solid red"> </hr>  
Question 1:  
In the paragraph-aware chunking method above, we simply split Wikipedia text using the '.' delimiter to approximate sentence boundaries.  
Discuss whether this is an effective strategy for creating meaningful chunks in a RAG system.
Propose one or more improved chunking strategies that could better capture document structure — and you may include code snippets to justify or demonstrate your approach.  
<hr style="border:10px solid red"> </hr>  
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>

Simply splitting Wikipedia text using '.' delimiters is ineffective for meaningful RAG chunks because it ignores actual sentence boundaries (e.g., abbreviations like "Dr." break sentences) and document structure, potentially creating incoherent segments.

To improve this trategy we could also include '\n\n', '\n', ',' or ';' or ' ' to give the model more tailored separations. We could do this in a hierarchical way, first we try to split at paragraph breaks, then lines, then proper sentence boundaries (with space to avoid abbreviation issues).


<hr style="border:10px solid green"> </hr>
</font></h4>

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: Consider a scenario where you want to perform RAG on source code (e.g., Python files, Java classes) instead of natural language text. Would the chunking methods demonstrated above (character-based and sentence/paragraph based with boundaries) work effectively for code? Explain why or why not, and describe how you would approach chunking source code to maintain semantic coherence. What specific characteristics of code structure would you need to consider?

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 2: </b><br>

Standard chunking methods fail for source code because they dont take in account syntactic structure—character-based splitting may break function definitions, while sentence splitting (using '.') is irrelevant.

We must have to respect language-specific constructs like functions, classes, or logical blocks.
AST (Abstract Syntax Tree) parsing ensures semantic coherence by chunking at node boundaries. For Python, the ast module can isolate coherent unit.

<hr style="border:10px solid green"> </hr>
</font></h4>

In [9]:
# Saving chunks ---

# Flatten chunks into a new DataFrame
records = []
for _, row in df.iterrows():
    doc_id = row["id"]
    title = row["title"]
    url = row["url"]
    for i, chunk in enumerate(row["paragraph_chunks"]):
        records.append({
            "doc_id": doc_id,
            "title": title,
            "url": url,
            "chunk_id": f"{doc_id}_chunk_{i}",
            "chunk_text": chunk.strip()
        })

# Create the flattened chunks DataFrame
chunks_df = pd.DataFrame(records)
print(f"Total chunks: {len(chunks_df)}")
print(f"Average chunk length: {chunks_df['chunk_text'].apply(len).mean():.2f} characters\n")

# Show example
print("Example rows:")
display(chunks_df.head())

chunks_df.to_csv("wikipedia_chunks.csv", index=False)
print("Chunks saved to 'wikipedia_chunks.csv'")
print(f"Total chunks: {len(chunks_df)}")


Total chunks: 7551
Average chunk length: 289.68 characters

Example rows:


,doc_id,title,url,chunk_id,chunk_text
0,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_0,Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: bor...
1,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_1,Indrani along with Clarence Wijewardena and An...
2,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_2,"She has one elder sister, Mallika and one youn..."
3,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_3,Her sister Mallika has been singing since 1965...
4,64741026,Indrani Perera,https://en.wikipedia.org/wiki/Indrani%20Perera,64741026_chunk_4,"The couple has one daughter, Anjalie Charukesh..."


Chunks saved to 'wikipedia_chunks.csv'
Total chunks: 7551


## <b>Part II: Embedding</b>

After chunking our documents, the next step is to convert text chunks into vector representations (embeddings). These embeddings capture the semantic meaning of the text in a high-dimensional space, allowing us to measure similarity between chunks and queries mathematically.

We will use **sentence-transformers/all-MiniLM-L6-v2**, a compact and efficient embedding model that produces 384-dimensional embeddings for English text. This model offers a strong balance between performance and computational efficiency, making it well-suited for our RAG pipeline.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
Fill in the <code>embed()</code> function to encode text chunks and generate normalized embeddings using <code>sentence-transformers/all-MiniLM-L6-v2</code>.  
Then, apply it to all documents in <code>chunks_df</code> and store the results.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [10]:
# --- 2.3 Embedding Generation ---
# Load model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"Loaded embedding model: {model_name}")


# --- Define embedding function ---
def embed(text_list, doc_type="document"):
    """
    Encodes a list of texts and returns normalized embeddings.
    """
    encoded = tokenizer(
        [f"search_{doc_type}: {t}" for t in text_list],
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        output = model(**encoded)  # FILLED: forward pass through the model
        token_embeddings = output.last_hidden_state  # FILLED: extract last hidden state
        # Use mean pooling to aggregate token embeddings
        attention_mask = encoded['attention_mask']
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        pooled = torch.sum(token_embeddings * input_mask_expanded, 1)  # FILLED: sum weighted by attention mask
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        pooled = pooled / sum_mask  # Normalize by the sum of attention weights
        pooled = F.normalize(pooled, p=2, dim=1)  # FILLED: apply L2 normalization
    return pooled.cpu()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loaded embedding model: sentence-transformers/all-MiniLM-L6-v2


In [11]:

# --- Test with one example ---
sample_text = ["Artificial intelligence is transforming the world."]
sample_emb = embed(sample_text)
print(f"Sample embedding shape: {sample_emb.shape}")

# --- Apply to all chunks ---
print(f"\nGenerating embeddings for {len(chunks_df)} chunks...")

emb_list = []
for i in tqdm(range(0, len(chunks_df), 32)):
    batch = chunks_df["chunk_text"].iloc[i:i+32].tolist()
    emb = embed(batch, doc_type="document")
    emb_list.append(emb)

chunk_embeddings = torch.cat(emb_list, dim=0).numpy()
chunks_df["embedding"] = list(chunk_embeddings)

print("\nEmbeddings generated and added to DataFrame.")
print(chunks_df[["chunk_id", "title", "embedding"]].head())

Sample embedding shape: torch.Size([1, 384])

Generating embeddings for 7551 chunks...


100%|██████████| 236/236 [00:19<00:00, 12.26it/s]


Embeddings generated and added to DataFrame.
           chunk_id           title  \
0  64741026_chunk_0  Indrani Perera   
1  64741026_chunk_1  Indrani Perera   
2  64741026_chunk_2  Indrani Perera   
3  64741026_chunk_3  Indrani Perera   
4  64741026_chunk_4  Indrani Perera   

                                           embedding  
0  [-0.083336405, 0.00020871096, -0.01853555, -0....  
1  [-0.077711314, -0.00784878, -0.13712998, 0.031...  
2  [0.00074521045, -0.014844091, -0.022141706, -0...  
3  [-0.067996256, -0.029783592, -0.101412386, -0....  
4  [-0.09408034, 0.04756599, -0.04939983, 0.05124...  


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: In most retrieval systems, embeddings are represented as fixed-size vectors. Can you think of a way to design embeddings that can flexibly adjust their size or level of detail while still preserving meaningful similarity between representations? How could such an approach benefit Retrieval-Augmented Generation (RAG) systems in practice, particularly for improving efficiency or adapting to different computational budgets?




Matryoshka Representation Learning (MRL) allows flexible embeddings, it does not have a fixed size embedding but

provides an elegant solution for flexible embeddings by learning MRL explicitly optimizes the first m dimensions (where m is chosen from a logarithmic set like {8, 16, 32, ..., 2048}) to be independently useful representations.
Like this lower dimensions capture essential semantic information while higher dimensions add finer details.



For RAG systems, this approach offers significant benefits: during retrieval, systems can use smaller embedding dimensions for initial candidate shortlisting (dramatically reducing computational cost), then progressively use higher dimensions for re-ranking only the most promising candidates.

In particular, the paper shows that it allows up to 14× theoretical speedups and 14× real-world wall-clock time improvements for large-scale retrieval, maintaining comparable accuracy to full-dimensional retrieval.

Thus, ths approach would allow RAG systems to adapt to different computational cost available, leveraging or not the full dimensionality, we get a good performance from lower dimensionality and all from a single trained model without multiple forward passes or storing separate embedding databases



<hr style="border:10px solid red"> </hr>
<i>Hint:</i> You can find the answer in the paper <a href="https://arxiv.org/pdf/2205.13147" target="_blank">Matryoshka Representation Learning</a>.
</font></h4>


### **Building a Simple Vector Database**

After generating embeddings for all our document chunks, the next step is to **store** them in a structure that allows fast similarity search.  
In this section, we will build a **simple in-memory vector database** using PyTorch tensors.  
Each entry in the database will correspond to a text chunk and its embedding, enabling efficient retrieval based on vector similarity.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Fill in the code to populate the database by computing embeddings for all text chunks in <code>chunks_df</code> using the <code>embed()</code> function.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [12]:
def populate_database(chunks_df, batch_size=16):
    """
    Populates a vector database from precomputed chunks_df.
    """
    n_chunks = len(chunks_df)

    sample_emb = embed([chunks_df["chunk_text"].iloc[0]])  # FILLED: compute one sample embedding
    output_dim = sample_emb.shape[1]  # FILLED: extract embedding dimension from sample

    vectorial_database = torch.zeros((n_chunks, output_dim), dtype=torch.float32)  # FILLED: initialize empty tensor
    chunk_list = chunks_df["chunk_text"].tolist()

    print(f"Populating vector database with {n_chunks} chunks...")

    n = 0
    for i in range(0, n_chunks, batch_size):
        batch = chunks_df["chunk_text"].iloc[i:i+batch_size].tolist()  # FILLED: select batch of chunk texts
        embeddings = embed(batch, doc_type="document")  # FILLED: compute embeddings for current batch
        vectorial_database[n:n + len(batch)] = embeddings  # FILLED: store embeddings in the tensor
        n += len(batch)

    return chunk_list, vectorial_database

In [13]:
#Build Vector Database
chunk_list, vectorial_database = populate_database(chunks_df)

print("\n✅ Vector database successfully built.")
print(f"Total stored chunks: {len(chunk_list)}")
print(f"Database tensor shape: {tuple(vectorial_database.shape)}")

Populating vector database with 7551 chunks...

✅ Vector database successfully built.
Total stored chunks: 7551
Database tensor shape: (7551, 384)


In [14]:
#Save vector databse
os.makedirs("vector_db", exist_ok=True)

# Save tensor + chunk list
torch.save(vectorial_database, "vector_db/vectorial_database.pth")

with open("vector_db/chunk_list.json", "w", encoding="utf-8") as f:
    json.dump(chunk_list, f, indent=4, ensure_ascii=False)

print("✅ Saved:")
print(" - vector_db/vectorial_database.pth")
print(" - vector_db/chunk_list.json")

✅ Saved:
 - vector_db/vectorial_database.pth
 - vector_db/chunk_list.json


In [15]:
# Load the database
vectorial_database = torch.load("vector_db/vectorial_database.pth", map_location=device)
vectorial_database.requires_grad_(False)

with open("vector_db/chunk_list.json", "r", encoding="utf-8") as f:
    chunk_list = json.load(f)

print(f"✅ Loaded {len(chunk_list)} chunks.")
print(f"Database shape: {vectorial_database.shape}\n")

# Inspect first few entries
for i, embedding_vector in enumerate(vectorial_database[:5]):
    print(f"Vector {i} → {embedding_vector[:5]}")
    print(f"Text snippet: {chunk_list[i][:300]}\n")


✅ Loaded 7551 chunks.
Database shape: torch.Size([7551, 384])

Vector 0 → tensor([-0.0833,  0.0002, -0.0185, -0.0393, -0.0240], device='cuda:0')
Text snippet: Indrani Perera (Sinhala:ඉන්ද්‍රානි පෙරේරා: born 15 February), is a Sri Lankan singer and playback singer

Vector 1 → tensor([-0.0777, -0.0078, -0.1371,  0.0313, -0.0627], device='cuda:0')
Text snippet: Indrani along with Clarence Wijewardena and Annesley Malewana are known as "The Original Sinhala Pop Trio".

Early life 
She was born on 15 February in Borella, and is the second of three girls in the family. Her father, Abeypala Perera was a Buddhist and mother, Muriel Perera was a Christian

Vector 2 → tensor([ 0.0007, -0.0148, -0.0221, -0.0279, -0.0702], device='cuda:0')
Text snippet: She has one elder sister, Mallika and one younger sister, Iranganie. Indrani studied at  Presbyterian Girls School in Regent Street. She studied Kandyan Dancing in the school

Vector 3 → tensor([-0.0680, -0.0298, -0.1014, -0.0095, -0.0971], device=

#### **Defining Similarity Metrics for Retrieval**

After populating our vector database with embeddings, the next step in a RAG pipeline is to define a *similarity metric* to measure how close two vectors are in the embedding space.  
Common metrics include **dot product**, **L2 distance**, and **cosine similarity**.  

In most retrieval systems, cosine similarity is preferred because it measures the *angle* between two vectors rather than their magnitude, allowing comparison based purely on semantic direction instead of scale.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Fill in the code to implement the <code>cosine_similarity()</code> function.  
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [16]:
def cosine_similarity(query_embeddings, doc_embeddings):
    """
    Computes cosine similarity between query and document embeddings
    using manual normalization.
    """
    query_magnitudes = torch.norm(query_embeddings, p=2, dim=1, keepdim=True)  # FILLED: calculate L2 norm for queries
    normalized_queries = query_embeddings / query_magnitudes  # FILLED: normalize queries by their magnitudes

    doc_magnitudes = torch.norm(doc_embeddings, p=2, dim=1, keepdim=True)  # FILLED: calculate L2 norm for documents
    normalized_docs = doc_embeddings / doc_magnitudes  # FILLED: normalize documents by their magnitudes

    similarity_matrix = torch.mm(normalized_queries, normalized_docs.T)  # FILLED: dot product between normalized vectors

    return similarity_matrix


# --- Example test ---
query_embeddings = embed([
    "What is t-SNE?",
    "Who is Laurens van der Maaten?"
], "query")

doc_embeddings = embed([
    "t-SNE is a dimensionality reduction algorithm created by Laurens van der Maaten."
], "document")

with torch.no_grad():
    sim_cos = cosine_similarity(query_embeddings, doc_embeddings)

print("🔍 Example cosine similarity scores:\n", sim_cos)

🔍 Example cosine similarity scores:
 tensor([[0.6621],
        [0.4333]])


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
Fill in the code to complete the <code>retrieve()</code> function.  
It should encode the input query using <code>embed()</code>, compute similarity with all vectors in <code>vectorial_database</code>, and return the top-<i>k</i> most similar text chunks.
<hr style="border:10px solid blue"> </hr>
</font></h4>


In [17]:
def retrieve(query,
             vectorial_database=vectorial_database,
             chunk_list=chunk_list,
             topk=5,
             verbose=False):
    """
    Retrieves top-k most similar chunks to a query using dot-product similarity.
    """
    with torch.no_grad():
        query_embedding = embed([query], "query")  # FILLED: encode input query
        query_embedding = query_embedding.to(device)  # FILLED: move to correct device
        similarity_scores = torch.mm(query_embedding, vectorial_database.T)  # FILLED: compute similarity (dot product)
        topk_results = torch.topk(similarity_scores, k=topk, dim=1)  # FILLED: extract top-k scores and indices

        if verbose:
            for score, idx in zip(topk_results.values[0], topk_results.indices[0]):
                print(f"\nScore: {score:.4f}")
                print(f"Chunk:\n{chunk_list[idx][:500]}\n{'-'*80}")

        retrieved_chunks = [chunk_list[idx] for idx in topk_results.indices[0]]  # FILLED: select top-k chunks
        return "\n\n".join(retrieved_chunks)  # FILLED: return concatenated chunks as single string

# Example query
query = "When was Luigi Boria born?" #Try different queries based on the documents in the wikipedia dataset
result = retrieve(query, topk=3, verbose=True)


Score: 0.6371
Chunk:
Luigi Boria (born April 23, 1958) is a Venezuelan-born American politician who served as mayor of Doral, Florida from 2012 to 2016
--------------------------------------------------------------------------------

Score: 0.5401
Chunk:
(1675–1684)
 Sede vacante (March 1684–August 1686)
 Nicolò Caranza (1686–1697)
 Giulio Della Rosa (1698–1699)
 Alessandro Roncoveri (1700–1711)
 Adriano Sermattei (1713–1719)
 Gherardo Zandemaria (1719–1731)
 Severino Antonio Missini (1732–1753)
 Girolamo Bajardi (1753–1775)
 Alessandro Garimberti (1776–1813)
Sede vacante (1813–1817)
 Aloisio San Vitale (1817–1836)
 Giovanni Tommaso Neuschel (1836–1843)
 Pier Grisologo Basetti (1843–1857)
Sede vacante (16 June 1857 – 20 June 1859)
 Francesco Ben
--------------------------------------------------------------------------------

Score: 0.5355
Chunk:
Since 2016, Boria has been a member of the Miami-Dade Beacon Council.

References 

American people of Italian descent
Venezuelan emigrants 

In [18]:
# Example query
query = "When was Luigi Boria born?" #Try different queries based on the documents in the wikipedia dataset
result = retrieve(query, topk=3, verbose=True)



Score: 0.6371
Chunk:
Luigi Boria (born April 23, 1958) is a Venezuelan-born American politician who served as mayor of Doral, Florida from 2012 to 2016
--------------------------------------------------------------------------------

Score: 0.5401
Chunk:
(1675–1684)
 Sede vacante (March 1684–August 1686)
 Nicolò Caranza (1686–1697)
 Giulio Della Rosa (1698–1699)
 Alessandro Roncoveri (1700–1711)
 Adriano Sermattei (1713–1719)
 Gherardo Zandemaria (1719–1731)
 Severino Antonio Missini (1732–1753)
 Girolamo Bajardi (1753–1775)
 Alessandro Garimberti (1776–1813)
Sede vacante (1813–1817)
 Aloisio San Vitale (1817–1836)
 Giovanni Tommaso Neuschel (1836–1843)
 Pier Grisologo Basetti (1843–1857)
Sede vacante (16 June 1857 – 20 June 1859)
 Francesco Ben
--------------------------------------------------------------------------------

Score: 0.5355
Chunk:
Since 2016, Boria has been a member of the Miami-Dade Beacon Council.

References 

American people of Italian descent
Venezuelan emigrants 

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 4: There are retrieval methods like BM25 that rely on lexical overlap between the query and documents, and others based on dense embeddings that capture semantic similarity beyond exact word matches. Explain how these two approaches differ in how they represent and compare text. Then, discuss how a hybrid retrieval strategy combining both can overcome their respective limitations and improve retrieval performance in RAG systems. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 4: </b><br>

BM25 relies on lexical overlap using statistical word frequency and document length normalization, representing text as sparse bag-of-words vectors and excelling at keyword matching but failing with semantic variations.

 Dense embeddings capture semantic meaning through neural networks, representing text as dense vectors in continuous space and enabling synonym and conceptual matching but struggling with rare terms and exact phrase matches.

Hybrid retrieval combines both approaches by using BM25 for precise keyword recall and dense embeddings for semantic understanding, overcoming individual limitations through techniques like reciprocal rank fusion to leverage both lexical and semantic signals, significantly improving RAG performance by ensuring both precise term matching and conceptual relevance.


<hr style="border:10px solid green"> </hr>
</font></h4>


In real-world RAG systems, instead of manually storing and comparing vectors, we rely on **vector databases** such as **ChromaDB**, which are optimized for efficient **similarity search**, **indexing**, and **retrieval** at scale.  

These databases provide:
- Fast nearest-neighbor search (e.g., using HNSW graphs)  
- Persistent storage for millions of embeddings  
- Built-in support for different similarity metrics (cosine, L2, inner product)  


In [19]:
import chromadb
from chromadb.config import Settings

# --- Initialize ChromaDB client ---
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Reset ensures a clean state
chroma_client.reset()

# --- Create a collection ---
# You can choose the similarity metric: "cosine", "l2", or "ip" (inner product)
collection_name = "wikipedia_chunks"
collection = chroma_client.create_collection(
    name=collection_name,
    metadata={"hnsw:space": "cosine"}  # cosine similarity works best for normalized embeddings
)

print(f"✅ Created collection: {collection_name}")


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ Created collection: wikipedia_chunks


In [20]:
# Prepare the data
ids = chunks_df["chunk_id"].tolist()
embeddings = chunks_df["embedding"].tolist()
documents = chunks_df["chunk_text"].tolist()

# Ensure all embeddings are plain Python lists
embeddings = [e.tolist() if hasattr(e, "tolist") else e for e in embeddings]

# Add useful metadata for inspection
metadatas = [
    {
        "doc_id": row["doc_id"],
        "title": row["title"],
        "url": row["url"]
    }
    for _, row in chunks_df.iterrows()
]

# Add to the collection
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas
)

print(f"\n✅ Added {collection.count()} chunks to the collection")
print(f"Collection metadata: {collection.metadata}")


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given



✅ Added 7551 chunks to the collection
Collection metadata: {'hnsw:space': 'cosine'}


In [21]:
# Encode the query using our embed() function
query = "When was Luigi Boria born?"
query_embedding = embed([query], doc_type="query")[0].tolist()  # get single vector as list

# Query the collection
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3 # number of retrieved results
)

# Display results
print(f"🔎 Query: {query}\n" + "=" * 80)
for i in range(len(results["documents"][0])):
    print(f"\nResult {i+1}:")
    print(f"Title: {results['metadatas'][0][i]['title']}")
    print(f"Similarity score: {1 - results['distances'][0][i]:.4f}")  # cosine distance → similarity
    print(f"Text: {results['documents'][0][i][:300]}...")
    print("-" * 80)


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


🔎 Query: When was Luigi Boria born?

Result 1:
Title: Roman Catholic Diocese of Fidenza
Similarity score: 0.5401
Text: (1675–1684)
 Sede vacante (March 1684–August 1686)
 Nicolò Caranza (1686–1697)
 Giulio Della Rosa (1698–1699)
 Alessandro Roncoveri (1700–1711)
 Adriano Sermattei (1713–1719)
 Gherardo Zandemaria (1719–1731)
 Severino Antonio Missini (1732–1753)
 Girolamo Bajardi (1753–1775)
 Alessandro Garimberti (...
--------------------------------------------------------------------------------

Result 2:
Title: Roman Catholic Diocese of Fidenza
Similarity score: 0.4675
Text: Bishop Mario Zanchin (1962–1988) conducted a diocesan synod in 1987.

Bishops of Borgo San Donnino

 Papirio Picedi (1603–1606)
 Giovanni Linati (1606–1620)
 Alfonso Pozzi (1620–1626) 
 Ranuccio Scotti Douglas (1627–1650)
 Filippo Casoni (1650–1659)
 Alessandro Pallavicini, O.S.B...
--------------------------------------------------------------------------------

Result 3:
Title: Roman Catholic Diocese of Fide

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 5: </b><br>
ChromaDB and other vector databases often rely on Hierarchical Navigable Small World (HNSW) graphs to perform efficient approximate nearest neighbor search.  
Explain how the HNSW algorithm organizes data to enable fast and accurate retrieval in high-dimensional spaces.  
Why is this structure particularly effective for large-scale embedding collections compared to brute-force search?

<hr style="border:10px solid red"> </hr>
<i>Reference:</i> <a href="https://arxiv.org/pdf/1603.09320" target="_blank">Efficient and Robust Approximate Nearest Neighbor Search using Hierarchical Navigable Small World Graphs</a>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 5: </b><br>


HNSW allows to reduce large-scale embedding collections because it reduces search complexity from O(n) in brute-force to O(log n) maintaining a good accuracy. This is thanks to the small world properties (structure created by the hierarchical graph embeddings) and the hierarchical navigation leading to nearest neighbors without comparing against all vectors.

The hierarchical graph structure is a set of layers where high layers have a lot of connections but not so much nodes and low layers have a lot of nodes but with short range connection.


<hr style="border:10px solid green"> </hr>
</font></h4>


### <b> Two-Stage Retrieval: Dense Retrieval + Reranker</b>

In RAG, the first retrieval step often returns passages that are similar in meaning but not always the most relevant.  
A **reranker** fixes this by re-evaluating the top retrieved chunks using a stronger model that jointly reads the query and each document to assign a more accurate relevance score.


In [23]:
# Clean installation with compatible versions
!pip install --quiet "numpy<2.0" pandas==2.2.2
!pip install --quiet transformers==4.36.2 datasets==2.21.0 torch==2.9.0
!pip install --quiet sentence-transformers==2.2.2
!pip install --quiet chromadb==0.4.22

# Now import
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import CrossEncoder
import chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [24]:
from sentence_transformers import CrossEncoder

# Load reranker model
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANKER_MODEL_NAME)

def retrieve_with_reranker(query, collection, initial_k=5, final_k=3):
    """
    Retrieves and reranks candidate documents for a given query.
    Returns both the initial dense results and reranked results.
    """
    query_embedding = embed([query], doc_type="query")[0].tolist()
    candidates = collection.query(query_embeddings=[query_embedding], n_results=initial_k)

    docs = candidates["documents"][0]
    metas = candidates["metadatas"][0]
    dense_scores = [(1 - s) for s in candidates["distances"][0]]

    pairs = [(query, d) for d in docs]
    ce_scores = reranker.predict(pairs)

    reranked = [
        {
            "title": metas[i].get("title", ""),
            "url": metas[i].get("url", ""),
            "text": docs[i],
            "dense_score": dense_scores[i],
            "rerank_score": float(ce_scores[i]),
        }
        for i in range(len(docs))
    ]
    reranked.sort(key=lambda x: x["rerank_score"], reverse=True)

    return docs, metas, dense_scores, reranked[:final_k]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [25]:
# Example usage
query = "Where did Luigi Boria study?" #Try other queries too
docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

# --- Display initial dense retrieval ---
print(f"\nInitial dense retrieval (Top 5):\n" + "=" * 80)
for i, (d, s, m) in enumerate(zip(docs, dense_scores, metas), 1):
    print(f"{i}. {m.get('title', '')}  |  Dense similarity: {s:.4f}")
    print(f"Text: {d[:300].replace('\n', ' ')}")
    print("-" * 80)

# --- Display top reranked results ---
print(f"\nAfter Cross-Encoder Reranking (Top 3):\n" + "=" * 80)
for i, item in enumerate(top_reranked, 1):
    print(f"{i}. {item['title']}")
    print(f"Dense similarity: {item['dense_score']:.4f} | Reranker score: {item['rerank_score']:.4f}")
    print(f"Text: {item['text'][:300].replace('\n', ' ')}")
    print("-" * 80)


Initial dense retrieval (Top 5):
1. Roman Catholic Diocese of Fidenza  |  Dense similarity: 0.5027
Text: (1660–1675)   Gaetano Garimberti C.R
--------------------------------------------------------------------------------
2. Roman Catholic Diocese of Pistoia  |  Dense similarity: 0.4850
Text: (in Latin)  (in Latin)   (in Latin)  Studies Beani, Gaetano (1912). La Chiesa pistoiese dalla sua origine ai tempi nostri. Appunti storici. Pistoia: D. Pagani 1912  [Chapter IV, pp. 41–85]   Lanzoni, Francesco (1927). Le diocesi d'Italia dalle origini al principio del secolo VII (an
--------------------------------------------------------------------------------
3. Arab American University (Palestine)  |  Dense similarity: 0.4767
Text: Among them was Dr
--------------------------------------------------------------------------------
4. Alexander Baumgartner  |  Dense similarity: 0.4732
Text: Gallen (city)
--------------------------------------------------------------------------------
5. Now Mus

![Bi-Encoder vs Cross-Encoder Architecture](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 6:  
The figure above compares a Bi-Encoder and a Cross-Encoder architecture.  
Rerankers such as <code>cross-encoder/ms-marco-MiniLM-L-6-v2</code> use the second approach, jointly encoding the query and document through a single transformer.  
Why does this joint encoding typically yield higher retrieval precision, and why is it applied as a second-stage reranker instead of being used directly for large-scale retrieval?
<hr style="border:10px solid red"> </hr>
</font></h4>


<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 6: </b><br>

Cross-Encoders yield higher retrieval precision because they jointly encode query-document pairs through a single transformer, enabling deep cross-attention that captures fine-grained semantic interactions and contextual relationships between specific terms in the query and document, unlike Bi-Encoders that process them separately and rely on cosine similarity of independent embeddings.

However, Cross-Encoders are applied as second-stage rerankers rather than for large-scale retrieval because their pairwise computation requires O(n) forward passes for n documents, making them computationally prohibitive for entire databases, whereas Bi-Encoders can precompute document embeddings once and perform efficient similarity search, creating an optimal pipeline where fast Bi-Encoders retrieve candidate documents and precise Cross-Encoders refine the final selection.
<hr style="border:10px solid green"> </hr>
</font></h4>


### **Integrating Retrieved Context into the LLM’s Prompt**

Now that we can retrieve and rerank the most relevant document chunks,  
we integrate them directly into the **language model’s prompt**.  
This step allows the model to **ground its answer on factual context** rather than relying solely on internal knowledge —  
thereby improving accuracy and reducing hallucinations.


In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

gen_model_name = "Qwen/Qwen2.5-0.5B-Instruct"
gen_tok = AutoTokenizer.from_pretrained(gen_model_name, trust_remote_code=True)
gen_model = AutoModelForCausalLM.from_pretrained(
    gen_model_name,
    device_map="auto",
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)

def generate(msg, max_new_tokens=128, temperature=0.2):
    messages = [{"role": "user", "content": msg}]
    inputs = gen_tok.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(gen_model.device)

    outputs = gen_model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        eos_token_id=gen_tok.eos_token_id,
        pad_token_id=gen_tok.pad_token_id if gen_tok.pad_token_id is not None else gen_tok.eos_token_id
    )

    return gen_tok.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True).strip()


# ============================================================
# NO-RAG vs WITH RAG
# ============================================================
query = "When was Luigi Boria born?" #Try other queries

print("ORIGINAL PROMPT\n" + "=" * 60)
print(query)

print("\nANSWER WITHOUT RAG\n" + "=" * 60)
print(generate(query))

docs, metas, dense_scores, top_reranked = retrieve_with_reranker(
    query=query,
    collection=collection,
    initial_k=5,
    final_k=3
)

context = "\n\n".join(h["text"] for h in top_reranked)[:1600]
rag_prompt = (
    f"Use only the context to answer. If unknown, say you don't know.\n\n"
    f"Context:\n{context}\n\n"
    f"Question: {query}\nAnswer:"
)

print("\nAUGMENTED PROMPT\n" + "=" * 60)
print(rag_prompt)

print("\nANSWER WITH RAG\n" + "=" * 60)
print(generate(rag_prompt))


ValueError: Tokenizer class Qwen2Tokenizer does not exist or is not currently imported.

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr> Question 7: In the RAG prompt construction step, we simply concatenate the retrieved chunks before the question. Discuss potential issues with this naive approach, such as token limits, redundancy, or irrelevant context dilution. Then, explain how we could select, weight, or summarize the retrieved chunks before injecting them into the prompt to improve generation quality and efficiency. <hr style="border:10px solid red"> </hr> </font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 7: </b><br>



The naive concatenation approach faces several critical issues degrading the generation quality:

It can exceed token limits causing truncation of valuable context

It includes redundant information from overlapping chunks,
Dilutes relevant content with irrelevant passages,
Fails to prioritize the most important information


.

 To improve this,

 we could mplement chunk selection by filtering low-confidence retrievals using score thresholds

 we could select weight by applying weighting mechanisms that prioritize chunks by similarity scores or positional importance

we could use extractive summarization to condense our information retrieving;, condensing non essential chunks.


<hr style="border:10px solid green"> </hr>
</font></h4>


#### **To go further**

- Experiment with other chunking methods (e.g., semantic or recursive chunking).  
- Explore **LangChain** and **LlamaIndex** for building modular RAG pipelines.  
- Try **hybrid retrieval** combining sparse (BM25) and dense embeddings.  
- Explore more advanced RAG methods such as **RAG-Fusion**, **Self-RAG**, and **Active-RAG**.  
- Experiment with **Matryoshka Representation Learning** for scalable embeddings.  
- Try **fine-tuning rerankers** or **retrievers** for domain-specific data.  
